In [ ]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install datasets
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import notebook_login

# Preprocessing

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_token = 100

In [ ]:
# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # gpt2 does not have default padding token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
def complete_conversation(row):
    return {"complete_conversation": "Question: " +row["question"] + "\nContext:" + row["context"] + "\nAnswer" + row["answer"] + "</s>"}

def tokenize_function(row):
    return tokenizer(row["complete_conversation"], max_length=max_token, truncation=False, padding="max_length")

def is_shorter_than_max_token(row):
    return len(row['input_ids']) <= max_token

In [ ]:
# Load and tokenize dataset
dataset = load_dataset("b-mc2/sql-create-context")
dataset = dataset['train'].train_test_split(test_size=0.05)

# Turn each row into one sentence
dataset = dataset.map(lambda x: complete_conversation(x))

# Tokenize dataset
dataset = dataset.map(lambda x: tokenize_function(x))

# Filter conversation longer than token limit
dataset = dataset.filter(is_shorter_than_max_token)

# Load model and preparing for training

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=bnb_config)
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)

In [ ]:
# LORA config
config = LoraConfig(
    r=16, 
    lora_alpha=32, #alpha scaling
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

# Training

In [ ]:
training_args = TrainingArguments(
    output_dir="output_dir",
    per_device_train_batch_size=20,
    gradient_accumulation_steps=5,
    num_train_epochs=1,
    learning_rate=1e-4,
    evaluation_strategy="epoch",
    warmup_steps=50,
    weight_decay=1e-3,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

# Inference

In [ ]:
def create_sql_query(question: str, context: str) -> str:
    input = "Question: " + question + "\nContext:" + context + "\nAnswer"
    
    # Encode and move tensor into cuda if applicable.
    encoded_input = tokenizer(input, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    output = model.generate(**encoded_input, max_new_tokens=256)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response[len(input):]
    return response

In [ ]:
create_sql_query("What is the highest age of users with name Danjie", "CREATE TABLE user (age INTEGER, name STRING)")

## Push the model to Huggingface

In [ ]:
model.push_to_hub("Danjie/SQLMaster", commit_message="first draft", private=False)

## Load the model from Huggingface

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Danjie/SQLMaster"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)